In [70]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=os.getenv("GEMINI_API_KEY")
)


In [71]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()
search_query = input("Search")

from langchain_community.utilities import GoogleSerperAPIWrapper

search = GoogleSerperAPIWrapper()
url = search.results(search_query)

result=search.run(search_query)

summary_prompt = PromptTemplate(
    input_variables=["text", "query"],
    template="""
You are a helpful research assistant.

Given this web content:
"{text}"

And user query:
"{query}"

Write a concise, useful summary answering the query based on the content. Use bullet points if relevant.
"""
)


summary_chain = LLMChain(llm=llm, prompt=summary_prompt)


response = summary_chain.run({
    "text": search_query,
    "query": result
})

title = url["organic"][0]["title"]
link = url["organic"][0]["link"]
snippet = url["organic"][0]["snippet"]

sitelink = url["organic"][0]["sitelinks"]

for link in sitelink:
    print(link["link"])

print(title)
print(link)
print(snippet)
print(response)

Search bengaluru


https://en.wikipedia.org/wiki/History_of_Bengaluru
https://en.wikipedia.org/wiki/Bengaluru_Urban_district
https://en.wikipedia.org/wiki/Bengaluru_Palace
https://en.wikipedia.org/wiki/Bengaluru_geography_and_environment
Bengaluru - Wikipedia
{'title': 'Geography', 'link': 'https://en.wikipedia.org/wiki/Bengaluru_geography_and_environment'}
Bengaluru, [a] also known as Bangalore (its official name until 1 November 2014), is the capital and largest city of the southern Indian state of Karnataka.
Bengaluru, also known as Bangalore (its official name until November 1, 2014), is:

*   The capital and largest city of the southern Indian state of Karnataka, a status it has held since 1830.
*   One of India's largest cities, with a population exceeding ten million, classifying it as a megacity.
*   Known as both the "Garden City" and "The Silicon Valley of India," and is considered a "techie's paradise."
*   A city with an active online community forum (r/bangalore) for local news, events, and 

In [126]:
import chromadb

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=api_key
)

embedded_text = embeddings.embed_query(result)

client = chromadb.CloudClient(
  api_key= os.getenv("CHROMA_API_KEY"),
  tenant=os.getenv("CHROMA_TENANT"),
  database=os.getenv("CHROMA_DATABASE"),
)

collection=client.get_or_create_collection(name="ai-browser")

doc_counter=0

doc_id = f"doc_{doc_counter}"
collection.add(
    documents=[result],
    embeddings=[embedded_text],
    metadatas=[{"source": url["organic"][0]["link"]}],
    ids=[doc_id]
)
doc_counter+=1

print("Result type:", type(result))
print("Embedded vector length:", len(embedded_text))
print("Metadata:", {"source": url})
print("Document added successfully!")


Result type: <class 'str'>
Embedded vector length: 768
Metadata: {'source': {'searchParameters': {'q': 'bengaluru', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Bengaluru - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Bengaluru', 'snippet': 'Bengaluru, [a] also known as Bangalore (its official name until 1 November 2014), is the capital and largest city of the southern Indian state of Karnataka.', 'sitelinks': [{'title': 'History of Bengaluru', 'link': 'https://en.wikipedia.org/wiki/History_of_Bengaluru'}, {'title': 'Bengaluru Urban district', 'link': 'https://en.wikipedia.org/wiki/Bengaluru_Urban_district'}, {'title': 'Bengaluru Palace', 'link': 'https://en.wikipedia.org/wiki/Bengaluru_Palace'}, {'title': 'Geography', 'link': 'https://en.wikipedia.org/wiki/Bengaluru_geography_and_environment'}], 'position': 1}, {'title': 'Bengaluru | History, Points of Interest, & Facts | Britannica', 'link': 'https://www.britannica.com/place/B

In [134]:
query = input("Question: ")
query_embedding = embeddings.embed_query(query)


results = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)


retrieved_text = results["documents"][0][0] if results.get("documents") else ""

summary_prompt = PromptTemplate(
    input_variables=["text", "query"],
    template="""
You are a helpful summarizer assistant.

Given this web content which is already searched and saved in the vector database:
"{text}"

And user query:
"{query}"

Write a concise, useful summary answering the query based on the content. Do not make up the answer, if you do not know the answer say "I don't know the answer!".
"""
)


summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

response = summary_chain.run({
    "text": retrieved_text,
    "query": query
})

doc_counter=0
def fetch_or_search(query, response, search):
    global doc_counter
    if not response.strip() or response.strip() == "I don't know the answer!":
        output=search.run(query)
        try:
            parsed = json.loads(output)
            link = parsed["organic"][0]["link"]
        except Exception:
            link = "N/A"
        try:
            output_embedding = embeddings.embed_query(output)
            doc_id = f"doc_{doc_counter}"
            collection.add(
                documents=[output],
                embeddings=[output_embedding],
                metadatas=[{"source": link}],
                ids=[doc_id]
            )
            doc_counter += 1
        except Exception as e:
            print("Error embedding or storing:", e)
        return output
    return response

output = fetch_or_search(query, response, search)


print("\nFinal Output:\n", output)


Question:  bengaluru food street



Final Output:
 In this video, I explore VV Puram Food Street in Bengaluru, India — a spot locals told me has some of the best street food in the city. Baa maga, eat street hogana, or some of the BDA complexes have good street food nearby. Church street street food is a scam tho. Upvote 12 Bangalore Food Street in Marathahalli,Banguluru. Thatte idly, vada and chutney Location: Marathahalli, @netra_cafe Rating: 4.5/5 Follow @bangalorefoodstreet ... A whole street dedicated to food lovers. All kinds of Indian cuisine can be found there. Right from Rajasthani, Gujarathi, Bengali, South Indian, Chinese, Chats ... Walk through Bangalore's most famous food street. Savour South Indian butte dosa, idli vada or the local favourite hollige. Taste North Indian chat items. The street has over 20 food stalls in less than 150 meters serving varieties of pure vegetarian street food, sweets, South Indian, North Indian and Chinese ... Come join us for mouth watering Bangalore Street Food Tours as visit